# AGQD Data

By Max Woolf

*This notebook is licensed under the MIT License. If you use the code or data visualization designs contained within this notebook, it would be greatly appreciated if proper attribution is given back to this notebook and/or myself. Thanks! :)*

In [141]:
options(warn = -1)

# IMPORTANT: This assumes that all packages in "Rstart.R" are installed,
# and the fonts "Source Sans Pro" and "Open Sans Condensed Bold" are installed
# via extrafont. If ggplot2 charts fail to render, you may need to change/remove the theme call.

source("Rstart.R")

sessionInfo()

R version 3.2.2 (2015-08-14)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: OS X 10.11.2 (El Capitan)

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] grid      stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] wordcloud_2.5      stringr_1.0.0      digest_0.6.8       RColorBrewer_1.1-2
[5] scales_0.3.0       extrafont_0.17     ggplot2_2.0.0      dplyr_0.4.3       
[9] readr_0.1.1       

loaded via a namespace (and not attached):
 [1] Rcpp_0.12.1      Rttf2pt1_1.3.3   magrittr_1.5     munsell_0.4.2   
 [5] uuid_0.1-2       colorspace_1.2-6 R6_2.1.1         plyr_1.8.3      
 [9] tools_3.2.2      parallel_3.2.2   gtable_0.1.2     DBI_0.3.1       
[13] extrafontdb_1.0  lazyeval_0.1.10  assertthat_0.1   IRdisplay_0.3   
[17] repr_0.4         base64enc_0.1-3  IRkernel_0.5     slam_0.1-32     
[21] evaluate_0.8     labeling_0.3     rzmq_0.7.7       stringi_0.5-5   
[25] j

## Clean up AGQD Data

In [142]:
df_donations <- read_csv("agdq_donations.csv")
df_comments <- read_csv("agdq_comments.csv")
df_runs <- read_csv("agdq_runs.csv")

### Fix donation data

In [143]:
print(df_donations %>% head())
nrow(df_donations)

Source: local data frame [6 x 7]

                                                 name.href
                                                     (chr)
1  https://gamesdonequick.com/tracker/donor/10249/agdq2016
2 https://gamesdonequick.com/tracker/donor/254740/agdq2016
3 https://gamesdonequick.com/tracker/donor/315713/agdq2016
4 https://gamesdonequick.com/tracker/donor/315712/agdq2016
5 https://gamesdonequick.com/tracker/donor/315709/agdq2016
6  https://gamesdonequick.com/tracker/donor/12859/agdq2016
Variables not shown: name.text (chr), time (chr), amount.href (chr),
  amount.text (chr), index (int), url (chr)


[1] 30528

In [144]:
getID <- function(x) {
    return(strsplit(x,"/")[[1]][6])
}

getID("https://gamesdonequick.com/tracker/donor/20093/agdq2016")

[1] "20093"

In [145]:
df_donations <- df_donations %>% mutate(id = sapply(amount.href, getID), donor_id = sapply(name.href, getID))

print(df_donations %>% select(id, donor_id) %>% head())

Source: local data frame [6 x 2]

      id donor_id
   (chr)    (chr)
1 246475    10249
2 246472   254740
3 246470   315713
4 246468   315712
5 246466   315709
6 246465    12859


Date Format is: `January 9th, 2016, 8:11:59 PM`

This is nonstandard and required hacky code to fix. You have been warned!

In [146]:
formatDate <- function(x) {
    tryCatch({
    day_split <- strsplit(x, " ")[[1]]
    day_part <- day_split[2]
    
    # if single digit, numeric component of day must have a leading zero
    day_leading <- sprintf("%02s",substring(day_part,1,nchar(day_part)-3))
    
    day_split[2] <- day_leading
    day_string_new <- paste(day_split, collapse=" ")
    day_date <- strptime(day_string_new, "%B %d %Y, %r")
    return (as.POSIXct(day_date))
    },
    error = function(x) {return(NA)})
}

formatDate("January 9th, 2016, 8:11:59 PM")

[1] "2016-01-09 20:11:59 PST"

In [147]:
df_donations <- df_donations %>% mutate(time_donated = sapply(time, formatDate))

print(df_donations %>% select(time_donated) %>% head())

Source: local data frame [6 x 1]

  time_donated
         (dbl)
1   1452413570
2   1452413218
3   1452412967
4   1452412725
5   1452412487
6   1452412452


In [148]:
getAmount <- function(x) {
    num <- gsub(",", "", x)
    return(as.numeric(substr(num,2,nchar(num))))
}

getAmount("$1,024.99")

[1] 1024.99

In [149]:
df_donations <- df_donations %>% mutate(amount_donated = sapply(amount.text, getAmount))

print(df_donations %>% select(amount_donated) %>% head())

Source: local data frame [6 x 1]

  amount_donated
           (dbl)
1             20
2             15
3              5
4            100
5             20
6              5


In [150]:
df_donations <- df_donations %>% select(id, donor=name.text, amount_donated, time_donated, donor_id)

print(df_donations %>% head())

Source: local data frame [6 x 5]

      id                 donor amount_donated time_donated donor_id
   (chr)                 (chr)          (dbl)        (dbl)    (chr)
1 246475           (Anonymous)             20   1452413570    10249
2 246472           (Anonymous)             15   1452413218   254740
3 246470           (Anonymous)              5   1452412967   315713
4 246468           (Anonymous)            100   1452412725   315712
5 246466           (Anonymous)             20   1452412487   315709
6 246465 A..., Narada (Narada)              5   1452412452    12859


### Fix Comments Data

Not much to be fixed fortunately.

In [151]:
print(df_comments %>% head())
nrow(df_comments)

Source: local data frame [6 x 4]

                                                                           comment
                                                                             (chr)
1 I'm lucky enough that I've never had to experience the pain of losing a loved on
2                                                                                 
3                                                                            Magoo
4 Been watching GDQ for years now and have always loved it. My grandpa was diagnos
5                                                                                 
6                                                                                 
Variables not shown: bid_category (chr), bid_choice (chr), url (chr)


[1] 29358

In [152]:
df_comments <- df_comments %>%
                    mutate(id = sapply(url, getID)) %>%
                    select(id, comment, bid_category = bid_category,
                          bid_choice = bid_choice)

print(df_comments %>% head())

Source: local data frame [6 x 4]

      id
   (chr)
1 208269
2 232076
3 215116
4 237071
5 239363
6 235921
Variables not shown: comment (chr), bid_category (chr), bid_choice (chr)


### Fix Run Data

In [153]:
print(df_runs %>% head())
nrow(df_runs)

Source: local data frame [6 x 7]

                               run_title.href
                                        (chr)
1 https://gamesdonequick.com/tracker/run/1295
2 https://gamesdonequick.com/tracker/run/1596
3 https://gamesdonequick.com/tracker/run/1597
4 https://gamesdonequick.com/tracker/run/1598
5 https://gamesdonequick.com/tracker/run/1599
6 https://gamesdonequick.com/tracker/run/1600
Variables not shown: run_title.text (chr), runner (chr), start_time (chr),
  end_time (chr), index (int), url (chr)


[1] 156

In [154]:
df_runs <- df_runs %>%
            mutate(start_time = sapply(start_time, formatDate), end_time = sapply(end_time, formatDate)) %>%
            select(run_title = run_title.text, start_time, end_time)

print(df_runs %>% head())

Source: local data frame [6 x 3]

                                   run_title start_time   end_time
                                       (chr)      (dbl)      (dbl)
1                              Agdq Pre-Show 1451838600 1451840400
2                              Splatoon Any% 1451840400 1451844480
3 Super Monkey Ball: Banana Blitz All Worlds 1451844480 1451848080
4    Super Monkey Ball Expert Through Master 1451848080 1451849820
5                        Rayman Legends Any% 1451849820 1451856900
6          Kirby 64: The Crystal Shards 100% 1451856900 1451861898


### Merge time!

In [157]:
df <- df_donations %>% left_join(df_comments, by="id") %>%
        select(id, donor_id, donor, amount_donated, time_donated, comment, bid_category, bid_choice)

# Fix a few problematic donations manually
df[which(df$id==212588),7:8] <- c("Super Mario Maker Custom Level Blind Race", "Mario Maker Hard Level race")
df[which(df$id==215656),7:8] <- c("The Legend Of Zelda: Majora's Mask 4p Co-Op 100%", "Blindforded Co-op Majora Fight")

print(df %>% head())

Source: local data frame [6 x 8]

      id donor_id                 donor amount_donated time_donated
   (chr)    (chr)                 (chr)          (dbl)        (dbl)
1 246475    10249           (Anonymous)             20   1452413570
2 246472   254740           (Anonymous)             15   1452413218
3 246470   315713           (Anonymous)              5   1452412967
4 246468   315712           (Anonymous)            100   1452412725
5 246466   315709           (Anonymous)             20   1452412487
6 246465    12859 A..., Narada (Narada)              5   1452412452
Variables not shown: comment (chr), bid_category (chr), bid_choice (chr)


In [158]:
whenRunDonated <- function(x) {
    # if donation is > start time and < end time, that was when the donation was made
    run_truth_table = (x > df_runs$start_time) & (x < df_runs$end_time)
    if (sum(run_truth_table, na.rm=T) > 0) {return (df_runs$run_title[which(run_truth_table)])}
    else {return (NA)}
    
}

whenRunDonated(1452389135)

[1] "Bloodborne Any%"

In [159]:
df <- df %>% mutate(run_during_donated = sapply(time_donated, whenRunDonated))

print(df %>% select(run_during_donated) %>% head())

Source: local data frame [6 x 1]

                        run_during_donated
                                     (chr)
1                                       NA
2                                       NA
3                                       NA
4 Final Fantasy Iv Any% No 64 Floor Glitch
5 Final Fantasy Iv Any% No 64 Floor Glitch
6 Final Fantasy Iv Any% No 64 Floor Glitch


Done!

In [160]:
df_write <- df
df_write <- df_write %>% mutate(time_donated = format(as.POSIXlt(time_donated, origin = "1970-01-01"), "%Y-%m-%d %H:%M:%S"))
df_write[is.na(df_write)] <- ''

write.csv(df_write, "agqd_2016_data.csv", row.names=F)

rm(df_write)

## Visualizations

Start with a cumulative dist.

In [185]:
df_cumsum <- df %>% select(time_donated, amount_donated) %>%
                filter(!is.na(amount_donated)) %>%
                arrange(time_donated) %>%
                mutate(time_donated = as.POSIXct(time_donated, origin = "1970-01-01"),
                    cumsum = cumsum(amount_donated),
                    day = format(time_donated, "%d"))

print(df_cumsum %>% head())
print(nrow(df_cumsum))

Source: local data frame [6 x 4]

         time_donated amount_donated cumsum   day
               (time)          (dbl)  (dbl) (chr)
1 2015-12-09 07:15:19              1      1    09
2 2015-12-27 17:12:16              1      2    27
3 2016-01-01 07:24:13              3      5    01
4 2016-01-02 12:34:44              5     10    02
5 2016-01-03 06:02:50              5     15    03
6 2016-01-03 06:07:04              5     20    03
[1] 30540


In [186]:
plot <- ggplot(df_cumsum, aes(x=time_donated, y=cumsum, color=day)) +
            geom_line() +
            fte_theme() +
            scale_x_datetime(limits=c(as.POSIXct("2016-01-03 00:00:00"), as.POSIXct("2016-01-10 00:00:00"))) +
            scale_y_continuous(label=dollar) +
            theme(axis.title.x = element_blank(), axis.title.y = element_blank()) +
            labs(title="Cumulative $ Amount Raised For Charity by AGDQ 2016")

max_save(plot, "agdq-1", "Games Done Quick")

![](agdq-1.png)

In [163]:
mean <- mean(df$amount_donated, na.rm=T)
median <- median(df$amount_donated, na.rm=T)
lte_100 <- sum(df$amount_donated <= 100, na.rm=T) / nrow(df)

print(mean)
print(median)
print(lte_100)

[1] 39.61902
[1] 20
[1] 0.9704342


In [164]:
plot <- ggplot(df, aes(x=amount_donated)) +
            geom_histogram(binwidth=1, fill="#2e8ece") +
            fte_theme() +
            scale_x_continuous(label=dollar, limits=c(0,101), breaks=seq(0,100, by=10)) +
            scale_y_continuous(label=comma) +
            labs(title="Histogram of 97% of AGDQ 2016 Donations", x="Donation Amount", y="# of Donations")

max_save(plot, "agdq-2", "Games Done Quick")

![](agdq-2.png)

In [165]:
twoLine <- function(x, minchar) {
    if (nchar(x) < minchar) {return (x)}
    
    middle_index <- round(nchar(x)/2)
    
    space_indices <- unlist(gregexpr(" ", x))
    replacement_index = space_indices[which(abs(space_indices-middle_index) == min(abs(space_indices-middle_index)))]

    left <- substr(x, 1, replacement_index-1)
    right <- substr(x, replacement_index+1, nchar(x))
    
    return(paste(left,right,sep='\n'))
    
}

df_bidrun <- df %>% group_by(run_during_donated) %>%
                summarize(count=n()) %>%
                arrange(desc(count)) %>%
                head(15) %>%
                mutate(run_during_donated = sapply(run_during_donated, twoLine,30)) %>%
                mutate(run_during_donated = factor(run_during_donated, levels=rev(run_during_donated)))
               

print(df_bidrun %>% head())

Source: local data frame [6 x 2]

                                                 run_during_donated count
                                                             (fctr) (int)
1                 The Legend Of Zelda:\nMajora's Mask 4p Co-Op 100%  1575
2                         Final Fantasy Iv Any%\nNo 64 Floor Glitch  1515
3                                 Super Metroid\nReverse Boss Order  1470
4       The Legend Of Zelda: A Link\nTo The Past Reverse Boss Order  1208
5 The Legend Of Zelda: Ocarina Of\nTime / Master Quest All Dungeons  1047
6                                     Super Mario Galaxy Any% Luigi   851


In [166]:
plot <- ggplot(df_bidrun, aes(x=run_during_donated, y=count)) +
            geom_bar(stat="identity", fill="#e74c3c") +
            coord_flip() +
            fte_theme() +
            geom_text(aes(label=format(count, big.mark=",")), color="white", hjust=1.5, family="Open Sans Condensed Bold", size=2.5) +
            theme(axis.title.y = element_blank(), axis.text.y = element_text(size=6)) +
            labs(y="Total Number of Donations during Run", title="AGDQ 2016 Runs Which Generated Most Donations")

max_save(plot, "agdq-3", "Games Done Quick", h=5)

![](agdq-3.png)

In [183]:
df_bidrun <- df %>% group_by(run_during_donated) %>%
                summarize(total_amount = sum(amount_donated, na.rm=T)) %>%
                arrange(desc(total_amount)) %>%
                head(15) %>%
                mutate(run_during_donated = sapply(run_during_donated, twoLine,30)) %>%
                mutate(run_during_donated = factor(run_during_donated, levels=rev(run_during_donated)))
               

print(df_bidrun %>% head())

Source: local data frame [6 x 2]

                                                 run_during_donated
                                                             (fctr)
1                         Final Fantasy Iv Any%\nNo 64 Floor Glitch
2                                 Super Metroid\nReverse Boss Order
3                 The Legend Of Zelda:\nMajora's Mask 4p Co-Op 100%
4       The Legend Of Zelda: A Link\nTo The Past Reverse Boss Order
5 The Legend Of Zelda: Ocarina Of\nTime / Master Quest All Dungeons
6                                 Tasbot Takes\nTotal Control Of...
Variables not shown: total_amount (dbl)


In [184]:
plot <- ggplot(df_bidrun, aes(x=run_during_donated, y=total_amount)) +
            geom_bar(stat="identity", fill="#2ecc71") +
            coord_flip() +
            fte_theme() +
            scale_y_continuous(labels=dollar) +
            geom_text(aes(label=paste0("$",format(round(total_amount), big.mark=","))), color="white", hjust=1.1, family="Open Sans Condensed Bold", size=2.5) +
            theme(axis.title.y = element_blank(), axis.text.y = element_text(size=6)) +
            labs(y="Total $ Donated During Run", title="AGDQ 2016 Runs Which Most $ Donated")

max_save(plot, "agdq-4", "Games Done Quick", h=5)

![](agdq-4.png)

In [187]:
df_bidcat <- df %>% group_by(bid_category) %>%
                filter(bid_category!='') %>%
                summarize(total_amount = sum(amount_donated, na.rm=T)) %>%
                arrange(desc(total_amount)) %>%
                head(10) %>%
                mutate(bid_category = sapply(bid_category, twoLine,30)) %>%
                mutate(bid_category = factor(bid_category, levels=rev(bid_category)))
           

print(df_bidcat %>% head())

Source: local data frame [6 x 2]

                                                 bid_category total_amount
                                                       (fctr)        (dbl)
1                           Super Metroid\nReverse Boss Order    315580.11
2                   Final Fantasy Iv Any%\nNo 64 Floor Glitch    171300.67
3                                             Bloodborne Any%     50619.02
4                  Super Mario Maker\nCustom Level Blind Race     39482.81
5 The Legend Of Zelda: A Link\nTo The Past Reverse Boss Order     36248.11
6                                 Tasbot Plays Brain Age X100     34746.61


In [188]:
left <- c(paste0("$",format(round(df_bidcat$total_amount[1:2]), big.mark=",")), rep('',8))
right <- c(rep('',2), paste0("$",format(round(df_bidcat$total_amount[3:10]), big.mark=",")))

plot <- ggplot(df_bidcat, aes(x=bid_category, y=total_amount)) +
            geom_bar(stat="identity", fill="#1abc9c") +
            coord_flip() +
            fte_theme() +
            scale_y_continuous(labels=dollar) +
            geom_text(label=left, color="white", hjust=1.1, family="Open Sans Condensed Bold", size=2.5) +
            geom_text(label=right, color="#1abc9c", hjust=-0.2, family="Open Sans Condensed Bold", size=2.5) +
            theme(axis.title.y = element_blank(), axis.text.y = element_text(size=6)) +
            labs(y="Total $ Donated Toward Incentive", title="Top AGDQ 2016 Donation Run Incentives")

max_save(plot, "agdq-5", "Games Done Quick")

![](agdq-5.png)

In [175]:
df_bidcat <- df %>% group_by(bid_choice) %>%
                filter(bid_choice!='') %>%
                summarize(total_amount = sum(amount_donated, na.rm=T)) %>%
                arrange(desc(total_amount)) %>%
                head(10) %>%
                mutate(bid_choice = sapply(bid_choice, twoLine,30)) %>%
                mutate(bid_choice = factor(bid_choice, levels=rev(bid_choice)))

#cat(df_bidcat[2,1])
               

print(df_bidcat %>% head())

Source: local data frame [6 x 2]

                                      bid_choice total_amount
                                          (fctr)        (dbl)
1         Save or Kill the\nAnimals -- Kill them    165885.12
2         Save or Kill the\nAnimals -- Save them    149694.99
3                                  Fight Bahamut     21898.07
4                           Twitch Helps Play...     21239.55
5                    Mario Maker Hard Level race     20292.92
6 Bonus Game - Pokemon\nBlue (Glitch Exhibition)     20022.25


In [176]:
left <- c(paste0("$",format(round(df_bidcat$total_amount[1:2]), big.mark=",")), rep('',8))
right <- c(rep('',2), paste0("$",format(round(df_bidcat$total_amount[3:10]), big.mark=",")))

plot <- ggplot(df_bidcat, aes(x=bid_choice, y=total_amount)) +
            geom_bar(stat="identity", fill="#9b59b6") +
            coord_flip() +
            fte_theme() +
            scale_y_continuous(labels=dollar) +
            geom_text(label=left, color="white", hjust=1.1, family="Open Sans Condensed Bold", size=2.5) +
            geom_text(label=right, color="#9b59b6", hjust=-0.2, family="Open Sans Condensed Bold", size=2.5) +
            theme(axis.title.y = element_blank(), axis.text.y = element_text(size=6)) +
            labs(y="Total $ Donated Toward Choice", title="Top AGDQ 2016 Donation Incentive Choices")

max_save(plot, "agdq-6", "Games Done Quick")

![](agdq-6.png)

In [179]:
stop_words <- unlist(strsplit("a,able,about,across,after,all,almost,also,am,among,an,and,any,are,as,at,be,because,been,but,by,can,cannot,could,dear,did,do,does,either,else,ever,every,for,from,get,got,had,has,have,he,her,hers,him,his,how,however,i,if,in,into,is,it,its,just,least,let,like,likely,may,me,might,most,must,my,neither,no,nor,not,of,off,often,on,only,or,other,our,own,rather,said,say,says,she,should,since,so,some,than,that,the,their,them,then,there,these,they,this,tis,to,too,twas,us,wants,was,we,were,what,when,where,which,while,who,whom,why,will,with,would,yet,you,your", ","))

words.list <- strsplit(tolower(as.character(df$comment)), split=" ")
words.vector <- unlist(words.list)
words.vector <- apply(as.array(words.vector),1,function (x) {return (gsub("[^\\w\\']","",x, perl=T))})
word.freq <- sort(table(words.vector),decreasing=TRUE)
word.freq <- word.freq[-which(names(word.freq) %in% stop_words)]
word.freq <- word.freq[which(word.freq>10)]

print(word.freq[1:10])

words.vector
 cancer animals    time    good   great     run    agdq    kill    love    save 
   5970    5607    4354    4309    4237    3718    3679    3333    3197    3121 


In [180]:
library(wordcloud)

pal <- brewer.pal(9, "Blues")
pal <- pal[-c(1:3)]   # Remove light colors

png(filename = "agdq-7.png", width = 1000, height = 1000, res= 300)

wordcloud(toupper(names(word.freq)),
          word.freq,
          scale=c(3,.1),
          random.order=F,
          rot.per=.10,
          max.words=5000,
          colors=pal,
          family="Source Sans Pro Bold",
          random.color=T)

dev.off()

pdf 
  2

![](agdq-7.png)

# The MIT License (MIT)

Copyright (c) 2016 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.